In [3]:
import py_entitymatching as em
import csv
import os
import pandas as pd
import py_entitymatching as em

# reading cleaned csv files from stage 2
A = em.read_csv_metadata('./filmcrave1.csv', key='id')
B = em.read_csv_metadata('./imdb1.csv', key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
# Creating instance for overlap blocker.
ob = em.OverlapBlocker()

In [5]:
# Overlapping on Title. (Atleast 1 word match) - [Performed better than 3-gram]
overlap_candidate = ob.block_tables(A, B, 'Title', 'Title', word_level=True, overlap_size=1, rem_stop_words=False, 
                    l_output_attrs=['Title', 'Overall Rating', 'Year', 'Genre', 'Directors', 'Actors'], 
                    r_output_attrs=['Title', 'Overall Rating', 'Year', 'Genre', 'Directors', 'Actors'],
                    show_progress=False)
len(overlap_candidate)

1139838

In [7]:
# Filtering previous stage blocker output further by matching atleast 1 word in directors
overlap_candidate1 = ob.block_candset(overlap_candidate, 'Directors', 'Directors', word_level=True, overlap_size=1, show_progress=False)
len(overlap_candidate1)

9459

In [8]:

# Creating instance for attribute blocker
ab = em.AttrEquivalenceBlocker()

# Blocking previous level blocker using matching year attribute
attribute_candidate = ab.block_candset(overlap_candidate1, 'Year', 'Year', show_progress=False)
len(attribute_candidate)

2024

In [9]:
attribute_candidate.to_csv('BlockingSurvivedDataset.csv')

In [10]:
dbg = em.debug_blocker(attribute_candidate, A, B, output_size = 10)

In [11]:
dbg

,_id,ltable_id,rtable_id,ltable_Title,ltable_Overall Rating,ltable_Genre,ltable_MPAA Rating,ltable_Directors,ltable_Actors,ltable_Plot,rtable_Title,rtable_Overall Rating,rtable_Genre,rtable_MPAA Rating,rtable_Directors,rtable_Actors,rtable_Plot
0,0,362,447,They Look Like People,2.90/4,Drama/Mystery,NR,Perry Blackshear,"MacLeod Andrews, Evan Dumouchel, Margaret Ying Drake","Suspecting that those around him are actually malevolent shape-shifters, a troubled man\n\nquest...",They Look Like People,6.0,"Drama, Horror, Mystery",Unrated,Perry Blackshear,"MacLeod Andrews,Evan Dumouchel,Margaret Ying Drake,Mick Casale","Suspecting that people around him are turning into evil creatures, a troubled man questions whet..."
1,1,573,562,Black Sea,2.62/4,Adventure/Thriller,R,Kevin Macdonald,"Jude Law, Scoot McNairy, Ben Mendelsohn","Trying to appease his former an employees, a submarine captain takes on a shady job for a shadow...",Black Sea,6.4,"Adventure, Drama, Thriller",R,Kevin Macdonald,"Jude Law,Scoot McNairy,Ben Mendelsohn,David Threlfall","In order to make good with his former employers, a submarine captain takes a job with a shadowy ..."
2,2,1434,1875,Beginners,2.87/4,Drama,R,Mike Mills,"Ewan McGregor, Christopher Plummer, M̩lanie Laurent",A young man must deal with two bombshells that have been dropped by his elderly father: that he ...,Beginners,7.2,"Comedy, Drama, Romance",R,Mike Mills,"Ewan McGregor,Christopher Plummer,M̩lanie Laurent,Goran Visnjic","A young man is rocked by two announcements from his elderly father: that he has terminal cancer,..."
3,3,1483,2280,Elite Squad: The Enemy Within ( Tropa de Elite 2 - O Inimigo Agora Outro ),2.74/4,Action/Crime,NR,Jos̩ Padilha,"Wagner Moura, Irandhir Santos, Andr̩ Ramiro","After a prison riot, Captain Nascimiento, now a high-ranking security officer in Rio de Janeiro,...",Elite Squad: The Enemy Within,8.1,"Action, Crime, Drama",Not Rated,Jos̩ Padilha,"Wagner Moura,Irandhir Santos,Andr̩ Ramiro,Milhem Cortaz","After a prison riot, former-Captain Nascimento, now a high ranking security officer in Rio de Ja..."
4,4,1661,2544,"Good, the Bad, the Weird, The ( Joheunnom nabbeunnom isanghannom )",2.87/4,Adventure/Comedy,NR,Ji-woon Kim,"Kang-ho Song, Byung-hun Lee, Woo-sung Jung",The story of three Korean outlaws in 1940s Manchuria and their rivalry to possess a treasure map...,The Good the Bad the Weird,7.3,"Action, Adventure, Western",R,Jee-woon Kim,"Kang-ho Song,Byung-hun Lee,Woo-sung Jung,Je-mun Yun",The story of two outlaws and a bounty hunter in 1940s Manchuria and their rivalry to possess a t...
5,5,684,1703,Ida,3.16/4,Drama,PG-13,Pawel Pawlikowski,"Agata Kulesza, Agata Trzebuchowska, Joanna Kulig","In 1960s Poland, a young woman is on the verge of taking her vows to become a nun when she uncov...",Ida,7.4,Drama,PG-13,Pawel Pawlikowski,"Agata Kulesza,Agata Trzebuchowska,Dawid Ogrodnik,Jerzy Trela","Anna, a young novitiate nun in 1960s Poland, is on the verge of taking her vows when she discove..."
6,6,2069,2071,Unthinkable,2.45/4,Drama/Thriller,R,Gregor Jordan,"Samuel L. Jackson, Michael Sheen, Brandon Routh",A psychological thriller centered around a black-ops interrogator and an FBI agent who press a s...,Unthinkable,7.1,"Drama, Thriller",R,Gregor Jordan,"Samuel L. Jackson,Carrie-Anne Moss,Michael Sheen,Stephen Root",A psychological thriller centered around a black-ops interrogator and an FBI agent who press a s...
7,7,2017,2717,Me and Orson Welles,2.68/4,Drama,PG-13,Richard Linklater,"Ben Chaplin, Claire Danes, Zac Efron","A teenager is cast in the Mercury Theatre production of ""Julius Caesar"" directed by a young Orso...",Me and Orson Welles,6.8,"Comedy, Drama",PG-13,Richard Linklater,"Zac Efron,Claire Danes,Christian McKay,Zoe Kazan","In 1937, a teenager is cast in the Mercury Theatre production of ""Julius Caesar"", directed by a ..."
8,8,2878,3425,Turtles Can Fly ( Lakposhtha parvaz mikonand ),2.93/4,Drama/War,PG-13,Bahman Ghobadi,"Soran Ebrahim, Avaz Latif, Saddam Hossein Feysa

In [19]:
# Randomly sampling 500 records from the candidate tuple pairs
sample = em.sample_table(attribute_candidate, 500)

In [20]:
# Storing this sample for labelling.
sample.to_csv("Sampleset.csv")

In [12]:
# Opening the csv file and cleaning and converting to new CSV File
with open('./Sampleset.csv', 'r', encoding='utf-8', errors='ignore') as infile, open('./Sampleset1.csv', 'w') as outfile:
     inputs = csv.reader(infile)
     outputs = csv.writer(outfile)

     for index, row in enumerate(inputs):
         outputs.writerow(row)

# Read in the labelled dataset        
G = em.read_csv_metadata("./Sampleset1.csv", key='_id', 
                         fk_ltable='ltable_id', fk_rtable='rtable_id',
                         ltable=A, rtable=B)

Metadata file is not present in the given path; proceeding to read the csv file.


In [13]:
# Splitting the data into training and testing portions. 
split = em.split_train_test(G, train_proportion=0.5, random_state=0)

# Training and testing set splits
train_set = split['train']
test_set = split['test']

In [14]:
print (len(train_set), len(test_set))

250 250


In [15]:
train_set.to_csv('Trainset.csv')
test_set.to_csv('Testset.csv')

In [16]:
# Initialising all ML algos
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [17]:
# Generating features for training
features = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [18]:
# Extracting feature vectors to train and create model
H = em.extract_feature_vecs(train_set, 
                            feature_table=features, 
                            attrs_after='label',
                            show_progress=False)

In [19]:
H.head()

,_id,ltable_id,rtable_id,id_id_exm,id_id_anm,id_id_lev_dist,id_id_lev_sim,Title_Title_jac_qgm_3_qgm_3,Title_Title_cos_dlm_dc0_dlm_dc0,Title_Title_jac_dlm_dc0_dlm_dc0,...,Directors_Directors_nmw,Directors_Directors_sw,Actors_Actors_jac_qgm_3_qgm_3,Actors_Actors_cos_dlm_dc0_dlm_dc0,Actors_Actors_mel,Actors_Actors_lev_dist,Actors_Actors_lev_sim,Plot_Plot_jac_qgm_3_qgm_3,Plot_Plot_cos_dlm_dc0_dlm_dc0,label
476,1007330,3001,3298,0,0.909945,3,0.250000,0.259259,0.408248,0.250000,...,15.0,15.0,0.585714,0.166667,0.898730,17.0,0.730159,0.053892,0.069171,1
162,350997,1110,1147,0,0.967742,2,0.500000,0.450000,0.816497,0.666667,...,12.0,12.0,0.537313,0.166667,0.871178,19.0,0.666667,0.216667,0.193601,1
34,63043,39,202,0,0.193069,3,0.000000,0.230769,0.408248,0.250000,...,12.0,12.0,0.557377,0.182574,0.881275,15.0,0.705882,0.124000,0.141069,1
44,85897,243,280,0,0.867857,2,0.333333,0.153846,0.707107,0.500000,...,12.0,12.0,0.558824,0.182574,0.882112,17.0,0.711864,0.216138,0.243778,1
97,168032,847,543,0,0.641086,2,0.333333,1.000000,1.000000,1.000000,...,13.0,13.0,0.346154,0.182574,0.667645,36.0,0.389831,0.321192,0.358209,1


In [20]:
# Checking if any value is null
any(pd.notnull(H))

True

In [21]:
# We found null values. Hence, used impute_table to fill up the other values with strategy - mean.
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

In [ ]:
# Running select matcher step to run all possible algos and pick the best ML
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)

In [24]:
# Showing cross validation statistics
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.989717,0.995122,0.992322
1,RF,1.000000,0.995122,0.997531
2,SVM,0.895191,1.000000,0.943919
3,LinReg,1.000000,1.000000,1.000000
4,LogReg,0.994595,0.995122,0.994791
5,NaiveBayes,1.000000,1.000000,1.000000


In [25]:
result['drill_down_cv_stats']['f1']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x12f70f780>,5,0.986301,0.987654,1.000000,0.987654,1.000000,0.992322
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x12f70f8d0>,5,1.000000,1.000000,1.000000,0.987654,1.000000,0.997531
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x13042c6a0>,5,0.923077,0.987654,0.909091,0.964706,0.935065,0.943919
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x12f70f9e8>,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x12f70f940>,5,0.986301,1.000000,1.000000,0.987654,1.000000,0.994791
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x125d1b908>,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
# Choosing Linear Regression as it has highest f1-score.
dt = em.DTMatcher(name='LinReg', random_state=0)

In [27]:
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

In [28]:
# Extracting feature vectors for test_set
L = em.extract_feature_vecs(test_set, feature_table=features,
                            attrs_after='label', show_progress=False)

In [29]:
# Checking for null values in test_Set
any(pd.notnull(L))

True

In [30]:
# Filling null values with mean of the column
L = em.impute_table(L, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

In [31]:
# Predicting on the test set
predictions = dt.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

In [32]:
predictions[['_id', 'ltable_id', 'rtable_id', 'predicted', 'proba']].head()

,_id,ltable_id,rtable_id,predicted,proba
90,155959,163,499,1,1.0
254,529437,1959,1736,1,1.0
283,603984,2180,1972,1,1.0
445,936841,3176,3076,1,1.0
461,971241,3124,3178,1,1.0


In [33]:
# Evaluating the matches and printing summary out
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 97.33% (182/187)
Recall : 98.38% (182/185)
F1 : 97.85%
False positives : 5 (out of 187 positive predictions)
False negatives : 3 (out of 63 negative predictions)


In [34]:
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')
classifiers = [dt, rf, ln, lg, nb]

for c in classifiers:
    c.fit(table=H, 
           exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
           target_attr='label')

    # Extracting feature vectors for test_set
    L = em.extract_feature_vecs(test_set, feature_table=features,
                                attrs_after='label', show_progress=False)
    # Checking for null values in test_Set
    any(pd.notnull(L))

    # Filling null values with mean of the column
    L = em.impute_table(L, 
                    exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                    strategy='mean')

    # Predicting on the test set
    predictions = c.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
                  append=True, target_attr='predicted', inplace=False, return_probs=True,
                            probs_attr='proba')

    predictions[['_id', 'ltable_id', 'rtable_id', 'predicted', 'proba']].head()

    # Evaluating the matches and printing summary out
    eval_result = em.eval_matches(predictions, 'label', 'predicted')
    em.print_eval_summary(eval_result)

Precision : 97.33% (182/187)
Recall : 98.38% (182/185)
F1 : 97.85%
False positives : 5 (out of 187 positive predictions)
False negatives : 3 (out of 63 negative predictions)
Precision : 97.87% (184/188)
Recall : 99.46% (184/185)
F1 : 98.66%
False positives : 4 (out of 188 positive predictions)
False negatives : 1 (out of 62 negative predictions)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
There is no proba function defined for Linear Regression Matcher in scikit learn. So we return the probs as 1


Precision : 98.4% (185/188)
Recall : 100.0% (185/185)
F1 : 99.2%
False positives : 3 (out of 188 positive predictions)
False negatives : 0 (out of 62 negative predictions)
Precision : 98.4% (185/188)
Recall : 100.0% (185/185)
F1 : 99.2%
False positives : 3 (out of 188 positive predictions)
False negatives : 0 (out of 62 negative predictions)
Precision : 98.4% (185/188)
Recall : 100.0% (185/185)
F1 : 99.2%
False positives : 3 (out of 188 positive predictions)
False negatives : 0 (out of 62 negative predictions)
